# First Practical Work

## Data Science and Engineering

### DESIGN OF AN IMAGE FILTER FUNCTION, PARALLELIZABLE AND SCALABLE

**Full name:** Daniel Toribio
    
**NIA:** 100454242

# **Introduction**

This first practical work consists on implementing a function that given an image and a filter, computes the filtered image. However, instead of following the standard procedure, we make use of all the processors and cores available in our computer, and parallelize the job. Therefore, the intial task, computing the filtered image, is splitted into smaller ones, called threads, which in this case correspond to computing one filtered row.

In order all of this practice, we make use of the multiprocessing package, which enables us to parallelize processes and exploit all the resources of our computers.


In [ ]:
# Needed libraries

import numpy as np
import multiprocessing as mp
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile
import time
from multiprocessing.sharedctypes import Value, Array, RawArray
from multiprocessing import Process, Lock
import ctypes

In [ ]:
# more needed libraries

import importlib
import myFunctions as my
import independent_module as ind

In [ ]:
importlib.reload(my)
importlib.reload(ind)

In [ ]:
# Here we choose which images we want

images=["chess.jpg","cloudsonjupi.jpg","digits.jpg","fence.jpg","hand-x-ray.jpg"]
F_IMAGE1=images[4] 
F_IMAGE2=images[4] 

In [ ]:
# Now we convert each image in an array and compute the size it has

image1= np.array(Image.open(F_IMAGE1))
data_buffer_size1=image1.shape[0]*image1.shape[1]*image1.shape[2]
image2= np.array(Image.open(F_IMAGE2))
data_buffer_size2=image2.shape[0]*image2.shape[1]*image2.shape[2]

In [ ]:
filter1=np.array([
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])
filter2=np.array([0.5, 0 , -0.5], ndmin=2)
filter3=np.array([[0.5],[0],[-0.5]])

filter4=np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]
])
filter5=np.array([
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]
])




In [ ]:
#We set the number of processors depending on the computer
NUMPROCESS=mp.cpu_count()


In [ ]:
# These are the memory spaces where the filtered images are going to be stored

filtered_image1_VECTOR=Array(ctypes.c_byte,data_buffer_size1) 
filtered_image2_VECTOR=Array(ctypes.c_byte,data_buffer_size2)

In [ ]:
def tonumpyarray(mp_arr):
    #mp_array is a shared memory array with lock
    
    return np.frombuffer(mp_arr.get_obj(),dtype=np.uint8)

# AFTER THIS CELL YOU HAVE TO WRITE YOUR CODE

After this cell you have to use the preloaded image defined in the F_IMAGE variable, select two of the 5 filter predefined, and using the NUMPROCESS processors, apply the filters to the image and check results.

* The first filter is impulse response filter (the image output must be equals to the original one).
* The second filter is an edge filter, first order in x axis,  
* The third filter is an edge filter, first order in y axis,
* the fourth filter is an edge filter, second order, bi-directional
* the fifth filter is a blur gausian filter.

In [ ]:
#YOUR CODE MUST BE START HERE
# EXERCISE 1
ind.image_filter(image1, filter2, NUMPROCESS, filtered_image1_VECTOR)

In [ ]:
ind.image_filter(image2, filter3, NUMPROCESS, filtered_image2_VECTOR)

In [ ]:
#YOUR CODE MUST END HERE

# Visualization
In this last cells we take the filtered images stored in the multiprocessing.Vector variables filtered_image1_VECTOR and filtered_image2_VECTOR, converts to numpy array, with the same shape of the orginal images, and show the results.

In [ ]:
filtered_image1=tonumpyarray(filtered_image1_VECTOR).reshape(image1.shape)
filtered_image2=tonumpyarray(filtered_image2_VECTOR).reshape(image2.shape)

In [ ]:
# create figure
fig = plt.figure(figsize=(14, 7))
# Adds a subplot at the 1st position
fig.add_subplot(1, 2, 1)
  
# showing image
plt.imshow(filtered_image1)
plt.axis('off')
plt.title("Filtered image 1")
  
# Adds a subplot at the 2nd position
fig.add_subplot(1, 2, 2)
  
# showing image
plt.imshow(image1)
plt.axis('off')
plt.title("Original image 1")

plt.show()

In [ ]:
# create figure
fig = plt.figure(figsize=(14, 7))
# Adds a subplot at the 1st position
fig.add_subplot(1, 2, 1)
  
# showing image
plt.imshow(filtered_image2)
plt.axis('off')
plt.title("Filtered image 2")
  
# Adds a subplot at the 2nd position
fig.add_subplot(1, 2, 2)
  
# showing image
plt.imshow(image2)
plt.axis('off')
plt.title("Original image 2")

plt.show()

## **Conclusions exercise 1**

In this first exercise we were asked to build a function that given an image, a filter, the number of processors and the shared space for the filtered image, it outputs the filtered result (it is also stored in the shared space). In order to parallelize this problem, we would divide this big task into smaller ones. In this case, each row of the image corresponds to a thread, that is assigned to a core of a processor. Our image_filter function is in charge of calling the edge_filter1 function that filters each row in parallel. In this procedure we make use of global variables, which can be accessed and modified by all threads. For instance, the shared matrix is the variable where each thread will store its corresponding filtered row. Nonetheless, in order to avoid race conditions and that threads modify the shared matrix at the same time, we need to use locks. By doing this, every time a new filtered row is being written by a thread, it is locked and cannot be accessed by the rest of them. 

In order to compute the filtered image, we have built the function, edge_filter1, that receives as input the row number to be filtered, writes the filtered row in the shared space and returns the filtered row. Then, since the image is stored as a global variable, it can be accessed by all threads, and since every thread receives a different row number, they will use different parts of the image. Then, since the filters can have different sizes, in order to simplify this issue we consider the most general case, when the filter size is 5x5, and then use just the pixels which corresponds to the actual filter size, and therefore not including the rest in the multiplication. Note that in the function we iterate through the dimensions of the image and the columns of the given row, so that each thread computes a complete row of the image. 

Finally, as we have explained, the resulting filtered row is stored in the global variable using locks. As it can be seen above, we plotted the result, and depending on which filter is being used, the output is similar to the original input, as when filter1 is used, or more different, as when filter2 is being used. We have also tried the execution without parallel processing and saw that when using the smallest image with the smallest filter there is not a lot of difference but once we start increasing the sizes of both, image and filter, we could see the importance of parallel processing as we save a lot of time.

In [ ]:
# EXERCISE 2

ind.filters_execution(image1, filter1, filter2, NUMPROCESS, filtered_image1_VECTOR, filtered_image2_VECTOR)

In [ ]:
filtered_image1=my.tonumpyarray(filtered_image1_VECTOR).reshape(image1.shape)
filtered_image2=my.tonumpyarray(filtered_image2_VECTOR).reshape(image2.shape)

# Visualization

In [ ]:
# create figure
fig = plt.figure(figsize=(14, 7))
# Adds a subplot at the 1st position
fig.add_subplot(1, 2, 1)
  
# showing image
plt.imshow(filtered_image1)
plt.axis('off')
plt.title("First filtered image")
  
# Adds a subplot at the 2nd position
fig.add_subplot(1, 2, 2)
  
# showing image
plt.imshow(filtered_image2)
plt.axis('off')
plt.title("Second filtered image")

plt.show()

## **Conclusions exercise 2**

In this second exercise, we were asked to execute the task of exercise 1 two times, so that two filters are applied to the same image at the same time. Those two processes need to be parallelized and executed synchronically. In order to do the latter, we first created the two processes that are going to be run, which in this case are the two filtering processes (same image with two filters). Then, by means of the start and join functions provided by the multiprocessing package, we were able to instantiate the two at the same time and parallelize them. The Process function assigns the function to be executed by each process, and passes the values of their parameters. Then, once the processes are defined, they start by calling the method start(). Then, to synchronize the execution, we have to wait until the two processes end using the method join(). Note that since two processes are being run at the same time, we had to duplicate all the functions built for exercise 1, so that the second process uses them, we have to allocate space to the second filtered image also. Finally, the two filtered images are displayed, and we can see that the result is not affected and the same as running each process separately.

# **Final conclusions**

Throughout this first practical work we have applied all the tools learnt in the first block of the Massive Computing course. Firstly, we have implemented parallelization in the filtering image task, making use of all the cores and processors of our computer. Then, we had to be careful with the global memory and how to access it, so that there are not race conditions. The latter was solved by means of locks. Therefore, we have made a very interesting project in which we have been able to reinforce all that has been learnt so far. 